In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from poisson_binomial import poisson_binomial_pmf

In [2]:
def poisson_binomial_pmf_easy(probs):
    probslen = probs.size
    result = np.zeros(probslen+1)
    poisson_binomial_pmf(probs,probslen,result)
    return result

p = np.array([0.3,0.6,0.2,0.1,0.9,0.5])
n = p.size
pmf = np.zeros(n+1)
poisson_binomial_pmf(p,n,pmf)
print(pmf,pmf.sum())
print(poisson_binomial_pmf_easy(p))

[0.01008 0.12388 0.3353  0.3484  0.153   0.02772 0.00162] 1.0
[0.01008 0.12388 0.3353  0.3484  0.153   0.02772 0.00162]


In [ ]:
from numba import njit

@njit
def poisson_binomial_likelihood(k,probs,probslen,pmf,subpmf,likelihood,gradient):
    
    # Compute the pmf
    poisson_binomial_pmf(probs,probslen,pmf)
    likelihood[0] = pmf[k]
    
    for i in range(probslen):
        p = probs[i]
        subpmf[0] = pmf[0]/(1.0-p)
        for j in range(probslen-1):
            subpmf[j] = (pmf[j]-subpmf[j-1]*p)/(1.0-p)
        subpmf[probslen-1] = pmf[probslen-1]/p
        gradient[i] = subpmf[k-1]-subpmf[k]
    

In [20]:
from numba import njit
            
@njit
def poisson_binomial_likelihood(k,probs,probslen,pmf,subpmf,likelihood,gradient):
    
    # Compute the pmf
    poisson_binomial_pmf(probs,probslen,pmf)
    likelihood[0] = pmf[k]
    
    
    if k == 0:
        gradient_first_term,gradient_second_term=0.0,1.0
    elif k == probslen:
        gradient_first_term,gradient_second_term=1.0,0.0
    else:
        gradient_first_term,gradient_second_term=1.0,1.0
    
    for i in range(probslen):
        
        p = probs[i]
        oneoveroneminusp = 1.0/(1.0-p)
        
        subpmf[0] = pmf[0]*oneoveroneminusp
        for j in range(1,probslen):
            subpmf[j] = (pmf[j]-subpmf[j-1]*p)*oneoveroneminusp
        subpmf[probslen-1] = pmf[probslen]/p
        
        gradient[i] = gradient_first_term*subpmf[k-1]-gradient_second_term*subpmf[k]
        
    

k = 5
subpmf = np.zeros(n)
gradient = np.zeros(n)
likelihood = np.zeros(1)
poisson_binomial_likelihood_new(k,p,n,pmf,subpmf,likelihood,gradient)

dp = 1e-4
print(likelihood[0])
print(poisson_binomial_pmf_easy(p+dp*np.array([0,0,0,1,0,0]))[k])
print(likelihood[0]+dp*gradient[3])

0.027719999999999998
0.02773152
0.02773152


In [15]:
%time poisson_binomial_likelihood_old(4,p,n,pmf,subpmf,likelihood,gradient)
%timeit poisson_binomial_likelihood_old(4,p,n,pmf,subpmf,likelihood,gradient)

CPU times: user 216 ms, sys: 7.58 ms, total: 223 ms
Wall time: 224 ms
The slowest run took 4.97 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 5: 1.12 µs per loop


In [16]:
%time poisson_binomial_likelihood_new(4,p,n,pmf,subpmf,likelihood,gradient)
%timeit poisson_binomial_likelihood_new(4,p,n,pmf,subpmf,likelihood,gradient)

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 11 µs
1000000 loops, best of 5: 1.08 µs per loop


In [ ]:
from numba import njit
            
@njit
def poisson_binomial_log_likelihood(k,probs,probslen,pmf,subpmf,log_likelihood,gradient):
    
    # Compute the pmf
    poisson_binomial_pmf(probs,probslen,pmf)
    likelihood = pmf[k]
    log_likelihood = np.log(likelihood)
    
    
    if k == 0:
        gradient_first_term,gradient_second_term=0.0,1.0
    elif k == probslen:
        gradient_first_term,gradient_second_term=1.0,0.0
    else:
        gradient_first_term,gradient_second_term=1.0,1.0
    
    for i in range(probslen):
        
        p = probs[i]
        oneoveroneminusp = 1.0/(1.0-p)
        
        subpmf[0] = pmf[0]*oneoveroneminusp
        for j in range(1,probslen):
            subpmf[j] = (pmf[j]-subpmf[j-1]*p)*oneoveroneminusp
        subpmf[probslen-1] = pmf[probslen]/p
        
        gradient[i] = (gradient_first_term*subpmf[k-1]-gradient_second_term*subpmf[k])/likelihood
        
    

k = 5
subpmf = np.zeros(n)
gradient = np.zeros(n)
log_likelihood = 0.0
poisson_binomial_likelihood_new(k,p,n,pmf,subpmf,likelihood,gradient)

dp = 1e-4
print(likelihood[0])
print(poisson_binomial_pmf_easy(p+dp*np.array([0,0,0,1,0,0]))[k])
print(likelihood[0]+dp*gradient[3])